In [ ]:
# PySpark: Spark's API for Python.
!pip install pyspark

In [ ]:
#import necessary packages 
from pyspark.sql import SparkSession

### What is SparkSession?
Spark Session: A unified entry point for DataFrame and Dataset APIs.It's object "spark" is default available in spark-shell and it can be created programmatically using SparkSession builder pattern.

In Apache Spark 2.x and later, the `SparkSession` is the entry point to any Spark functionality. When you want to run a Spark application, you first need to create a SparkSession. 

The `SparkSession.builder().getOrCreate()` method is a way to ensure that a SparkSession is created only once in an application.

`SparkSession.builder()`: This returns a SparkSession.Builder object, which is a builder for a SparkSession. With the builder, you can configure options for the SparkSession, such as appName, master, and various Spark configurations using the config method.

`getOrCreate()`: When called on a SparkSession.Builder object, this method: Retrieves the existing SparkSession if one already exists.
Creates a new SparkSession if none exists.

In [ ]:
#create spark session
spark = SparkSession.builder.getOrCreate()


### Create a DataFrame from a CSV file
`read`:
This is a method associated with SparkSession and it returns a DataFrameReader that can be used to read data. The read method provides functionality to read data from various sources into a Spark DataFrame.

`option('header', 'true')`:
The option method allows you to specify options when reading data. In this case, the option being set is 'header' with the value 'true'. This means that the first row of the CSV file (Traffic_Crashes_-_Crashes.csv) is considered as a header and will be used to name the columns of the DataFrame.

If this option wasn't set (or set to 'false'), the CSV file would be read without considering the first row as a header, and default column names would be assigned (like _c0, _c1, etc.).

**Q1: Load the data from the csv files into DataFrames.**

In [ ]:
# Load the data from the csv files into DataFrames.
crashes = spark.read.option('header', 'true').csv('Traffic_Crashes_-_Crashes.csv')
vehicles = spark.read.option('header' , 'true').csv('Traffic_Crashes_-_Vehicles.csv')
peoples = spark.read.option( 'header', 'true').csv('Traffic_Crashes_-_People.csv')


In [ ]:
# let's see what is the type of crashes
print(type(crashes))


In [ ]:
#let's see what is the data type of each DataFrame 
crashes.dtypes

In [ ]:
vehicles.dtypes


In [ ]:
peoples.dtypes

In PySpark, the `pyspark.sql.types` module provides a collection of data types that you can use to specify the schema of a DataFrame. When you're working with data in Spark, sometimes you might need to explicitly define or cast data to a specific type. This is where these imports come into play.

In [ ]:
from pyspark.sql. types import StringType 
from pyspark.sql. types import IntegerType

**Q2: Find the ratio of number of crashes where the person involved was using cell phone to that where the person was not using the cell phone.**

In [ ]:
peoples. groupby(peoples.DRIVER_ACTION).count().orderBy("count").show(n=50,truncate=False)

#### In PySpark, when working with DataFrames, the `.show()` method is used to display the rows of the DataFrame in a tabular format, primarily for visual inspection during development or debugging.

`truncate=False`: Content in each cell of the table will be displayed in full, regardless of its length. This means that if you have very long content in some cells, the display might stretch out horizontally, making it harder to read, but ensuring you see the full content.

In [ ]:
phone = peoples.groupby(peoples.DRIVER_ACTION).count().filter((peoples.DRIVER_ACTION == 'CELL PHONE USE OTHER THAN TEXTING') | (peoples.DRIVER_ACTION == 'TEXTING'))
phone.show(truncate=False)


In [ ]:
phone_crashes = phone.groupBy().sum('count').collect()[0][0]
print ("Crashes that occurs because of phone: ", phone_crashes)

In [ ]:
no_phone = peoples.groupby(peoples.DRIVER_ACTION).count().filter((peoples.DRIVER_ACTION != 'NONE') &(peoples.DRIVER_ACTION != 'UNKNOWN') & (peoples.DRIVER_ACTION != 'OTHER') & (peoples.DRIVER_ACTION != 'null') & (peoples.DRIVER_ACTION != 'CELL PHONE USE OTHER THAN TEXTING') &(peoples.DRIVER_ACTION != 'TEXTING'))
no_phone.show(truncate=False)
no_phone_crashes = no_phone.groupBy().sum('count').collect()[0][0]
print ("Crashes that occurs NOT because of phone: ", no_phone_crashes)

In [ ]:
print ("Ratio of phone to non-phone crashes",phone_crashes,"/",no_phone_crashes, "=" , (100 * (phone_crashes / no_phone_crashes))) 


**Q3: Find which three Age groups were involved with highest number of crashes.**

In [ ]:
from pyspark.sql.functions import col,isnull

In [ ]:
# To be able to do mathematicl comparison or operations the variavle type should be numeric (int, double, float,...)
# Here age is string which is text, we need to convert it to int first
peoples = peoples.withColumn("AGE", col("AGE").cast("integer"))
peoples.dtypes

In [ ]:
age_groups = peoples.filter(~isnull(col("AGE"))).groupBy("AGE").count().orderBy("count", ascending=False).limit(3)
age_groups.show()

**Q4. Find which month of the year has the highest crashes.**

In [ ]:
crashes.select("CRASH_DATE").show(n=100,truncate=False)

In [ ]:
from pyspark.sql.functions import month, dayofweek, to_date, substring

In [ ]:
crashes = crashes.withColumn("CRASH_DATE", to_date(substring(crashes["CRASH_DATE"], 1, 10), 'MM/dd/yyyy'))

crashes.dtypes


In [ ]:
crashes.select("CRASH_DATE").show(10, truncate=False)


In [ ]:

crashes_month = crashes.withColumn("Month", month(crashes["CRASH_DATE"])).groupBy("Month").count().orderBy("count", ascending=False).limit(1)
crashes_month.show()

**Q5. Find which day of the week has the least crashes.**

In the dayofweek function in PySpark, the days of the week are represented as integers from 1 (Sunday) to 7 (Saturday). <br>
1.Sunday <br>
2.Monday<br>
3.Tuesday<br>
4.Wednesday<br>
5.Thursday<br>
6.Friday<br>
7.Saturday<br>

In [ ]:
crashes_day = crashes.withColumn("Day", dayofweek(crashes["CRASH_DATE"])).groupBy("day").count().orderBy("count", ascending=False).limit(1)
crashes_day.show()